In [1]:
import sys
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
# run undersampled logistic on subset B to check predictions
# read/prep data
dat = pd.read_csv("data/tokenized_reviews.csv")
dat = dat.dropna()
dat["quote"] = dat["quote"].astype(int)
dat["tokenized_words"] = dat["tokenized_words"].apply(lambda x: x.strip("[']").replace("', '"," "))

# 85% train / 15% test
X_train, X_test, y_train, y_test = train_test_split(dat.drop(columns=["popular"]), 
                                                    dat["popular"],
                                                    test_size = 0.15,
                                                    random_state = 229)

# feature subsets
subset_b = ["user_reviews","days_since_review","user_rating","rating_diff",
            "num_words","avg_word_len","avg_sent_len","pct_verbs",
            "pct_nouns","pct_adj","quote","sentiment"]

# undersample train set
majority_size = len(y_train[y_train==0])
minority_size = len(y_train[y_train==1])
majority_indices = y_train[y_train==0].index
rng = np.random.default_rng(seed=229)
drop_indices = rng.choice(majority_indices, majority_size-minority_size, replace=False)
X_train = X_train.drop(drop_indices)
y_train = y_train.drop(drop_indices)

log_reg = sm.Logit(y_train, sm.add_constant(X_train[subset_b])).fit()
predictions = log_reg.predict(sm.add_constant(X_test[subset_b]))
predictions.rename("predictions", inplace=True)

Optimization terminated successfully.
         Current function value: 0.626418
         Iterations 6


1148226    0.345244
1898195    0.235176
1287911    0.382331
941466     0.715816
205890     0.597087
             ...   
1965816    0.439618
1848117    0.290346
1861785    0.541369
423898     0.331198
1063051    0.428067
Name: predictions, Length: 297156, dtype: float64

In [3]:
# merge test dataset with full set of features and predictions
raw = pd.read_csv("data/filtered_reviews.csv")
dat = X_test.merge(raw,left_index=True,right_index=True)
dat = dat.drop(columns=["user_reviews_x","days_since_review_x","user_rating_x"])
dat = dat.merge(predictions,left_index=True,right_index=True)

In [4]:
# error classification
conditions = [
    (dat["popular"]==0) & (dat["predictions"]<0.5),
    (dat["popular"]==0) & (dat["predictions"]>=0.5),
    (dat["popular"]==1) & (dat["predictions"]>=0.5),
    (dat["popular"]==1) & (dat["predictions"]<0.5)
]
values = ["TN","FP","TP","FN"]

dat["error"] = np.select(conditions, values)

In [5]:
# is prediction probability correlated with like share
dat[["like_share","predictions"]].corr()

,like_share,predictions
like_share,1.00000,0.29066
predictions,0.29066,1.00000


In [6]:
dat[dat["error"]=="FP"].describe()

,rating_diff,num_words,avg_word_len,avg_sent_len,pct_verbs,pct_nouns,pct_adj,quote,sentiment,book_id,user_reviews_y,user_rating_y,avg_rating,ratings_count,days_since_review_y,review_likes,like_share,popular,predictions
count,60103.000000,60103.000000,60103.000000,60103.000000,60103.000000,60103.000000,60103.000000,60103.000000,60103.000000,6.010300e+04,60103.000000,60103.000000,60103.000000,6.010300e+04,60103.000000,60103.000000,60103.000000,60103.0,60103.000000
mean,-0.080861,381.576261,4.335578,17.950723,0.190978,0.235747,0.164877,0.474302,0.161806,1.630731e+07,494.536146,3.820408,3.901269,7.960956e+04,1210.078249,4.167180,0.006466,0.0,0.630628
std,1.142228,302.091402,2.137312,9.499082,0.036652,0.059510,0.040450,0.499343,0.193815,9.989225e+06,564.368449,1.185833,0.301846,2.934422e+05,798.716740,8.855249,0.005390,0.0,0.116809
min,-4.560000,1.000000,2.500000,0.500000,0.000000,0.000000,0.000000,0.000000,-0.980900,1.000000e+00,1.000000,0.000000,1.250000,1.100000e+01,8.000000,1.000000,0.000030,0.0,0.500001
25%,-0.690000,162.000000,4.129310,13.718129,0.173684,0.207813,0.144030,0.000000,0.043612,8.542996e+06,138.000000,3.000000,3.730000,2.457000e+03,561.000000,1.000000,0.001783,0.0,0.539872
50%,0.110000,327.000000,4.290566,17.000000,0.193118,0.228989,0.163478,0.000000,0.153813,1.734732e+07,338.000000,4.000000,3.920000,9.374000e+03,1058.000000,2.000000,0.004942,0.0,0.594784
75%,0.810000,525.000000,4.475972,20.772727,0.211268,0.253575,0.183761,1.000000,0.271583,2.442363e+07,636.000000,5.000000,4.110000,3.995700e+04,1739.000000,4.000000,0.010309,0.0,0.687902
max,2.920000,3634.000000,452.000000,637.000000,1.000000,1.000000,1.000000,1.000000,0.999500,3.630724e+07,7938.000000,5.000000,4.910000,4.899965e+06,3883.000000,317.000000,0.020000,0.0,1.000000


In [7]:
dat[dat["error"]=="TP"].describe()

,rating_diff,num_words,avg_word_len,avg_sent_len,pct_verbs,pct_nouns,pct_adj,quote,sentiment,book_id,user_reviews_y,user_rating_y,avg_rating,ratings_count,days_since_review_y,review_likes,like_share,popular,predictions
count,34939.000000,34939.000000,34939.000000,34939.000000,34939.000000,34939.000000,34939.000000,34939.000000,34939.000000,3.493900e+04,34939.000000,34939.000000,34939.000000,3.493900e+04,34939.000000,34939.000000,34939.000000,34939.0,34939.000000
mean,-0.078254,405.555397,4.324690,17.801026,0.190426,0.235433,0.165279,0.484330,0.158231,1.726309e+07,694.443430,3.818856,3.897110,1.031054e+04,1188.701823,26.645811,0.125533,1.0,0.688236
std,1.174900,318.958835,0.289320,7.905548,0.032590,0.044196,0.036566,0.499762,0.187244,1.075835e+07,638.935198,1.223687,0.302119,5.376842e+04,848.572555,53.948399,0.170903,0.0,0.135583
min,-4.750000,1.000000,2.250000,0.500000,0.000000,0.000000,0.000000,0.000000,-0.996600,1.000000e+00,1.000000,0.000000,1.250000,1.100000e+01,7.000000,2.000000,0.020013,1.0,0.500007
25%,-0.680000,190.000000,4.140984,13.571429,0.172881,0.210232,0.144848,0.000000,0.041806,8.176075e+06,257.000000,3.000000,3.720000,5.810000e+02,495.000000,5.000000,0.032258,1.0,0.570976
50%,0.120000,340.000000,4.302789,16.869565,0.191429,0.231263,0.163462,0.000000,0.148593,1.816928e+07,513.000000,4.000000,3.920000,1.886000e+03,999.000000,11.000000,0.057851,1.0,0.664197
75%,0.830000,536.000000,4.486300,20.723180,0.209113,0.254723,0.183007,1.000000,0.268092,2.581703e+07,920.000000,5.000000,4.110000,6.165000e+03,1727.000000,29.000000,0.131649,1.0,0.787656
max,2.700000,3524.000000,10.500000,460.500000,0.666667,1.000000,1.000000,1.000000,0.994800,3.643911e+07,7938.000000,5.000000,5.000000,2.078406e+06,3882.000000,3394.000000,1.000000,1.0,0.999990


In [8]:
dat[dat["error"]=="TN"].describe()

,rating_diff,num_words,avg_word_len,avg_sent_len,pct_verbs,pct_nouns,pct_adj,quote,sentiment,book_id,user_reviews_y,user_rating_y,avg_rating,ratings_count,days_since_review_y,review_likes,like_share,popular,predictions
count,175840.000000,175840.000000,175840.000000,175840.000000,175840.000000,175840.000000,175840.000000,175840.000000,175840.000000,1.758400e+05,175840.000000,175840.000000,175840.000000,1.758400e+05,175840.000000,175840.000000,175840.000000,175840.0,175840.000000
mean,-0.057348,108.215343,4.213158,14.395314,0.201256,0.227458,0.185647,0.154322,0.231574,1.386399e+07,91.929726,3.989018,4.046367,1.733952e+05,1246.874852,2.508587,0.004352,0.0,0.368765
std,1.182021,103.928522,0.445662,7.239941,0.060151,0.064704,0.075612,0.361258,0.255378,1.023025e+07,107.871652,1.218944,0.259406,4.926486e+05,849.582512,5.158972,0.004841,0.0,0.069792
min,-5.000000,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,-0.988200,1.000000e+00,1.000000,0.000000,2.460000,3.000000e+00,0.000000,1.000000,0.000030,0.0,0.112211
25%,-0.530000,33.000000,3.963636,10.000000,0.170588,0.193857,0.145161,0.000000,0.066729,3.407877e+06,16.000000,3.000000,3.880000,4.953000e+03,575.000000,1.000000,0.000617,0.0,0.317740
50%,0.170000,73.000000,4.187970,13.600000,0.200000,0.224138,0.177083,0.000000,0.228454,1.363857e+07,51.000000,4.000000,4.060000,2.387000e+04,1061.000000,1.000000,0.002227,0.0,0.368828
75%,0.820000,151.000000,4.431373,17.750000,0.229008,0.257143,0.214286,0.000000,0.393789,2.250450e+07,128.000000,5.000000,4.230000,1.100500e+05,1775.000000,2.000000,0.006711,0.0,0.423173
max,2.190000,818.000000,12.000000,208.000000,1.000000,1.000000,1.000000,1.000000,0.998000,3.641325e+07,952.000000,5.000000,5.000000,4.899965e+06,4951.000000,483.000000,0.020000,0.0,0.500000


In [9]:
# load predictions from tfidf random forest
with open("data/rf_predictions.pkl", "rb") as fp:
    predictions = pickle.load(fp)

In [10]:
# merge test dataset with full set of features and predictions
raw = pd.read_csv("data/filtered_reviews.csv")
dat = X_test.merge(raw,left_index=True,right_index=True)
dat = dat.drop(columns=["user_reviews_x","days_since_review_x","user_rating_x"])
dat["predictions"] = predictions

In [11]:
# error classification
conditions = [
    (dat["popular"]==0) & (dat["predictions"]<0.5),
    (dat["popular"]==0) & (dat["predictions"]>=0.5),
    (dat["popular"]==1) & (dat["predictions"]>=0.5),
    (dat["popular"]==1) & (dat["predictions"]<0.5)
]
values = ["TN","FP","TP","FN"]

dat["error"] = np.select(conditions, values)

In [12]:
# is prediction probability correlated with like share
dat[["like_share","predictions"]].corr()

,like_share,predictions
like_share,1.000000,0.333259
predictions,0.333259,1.000000


In [13]:
dat[dat["error"]=="FP"].describe()

,rating_diff,num_words,avg_word_len,avg_sent_len,pct_verbs,pct_nouns,pct_adj,quote,sentiment,book_id,user_reviews_y,user_rating_y,avg_rating,ratings_count,days_since_review_y,review_likes,like_share,popular,predictions
count,67596.000000,67596.000000,67596.000000,67596.000000,67596.000000,67596.000000,67596.000000,67596.000000,67596.000000,6.759600e+04,67596.000000,67596.000000,67596.000000,6.759600e+04,67596.000000,67596.000000,67596.000000,67596.0,67596.000000
mean,-0.089392,290.684612,4.296775,17.059276,0.192704,0.234134,0.169701,0.365184,0.180965,1.639828e+07,441.398174,3.856959,3.946351,5.868199e+04,1216.049574,3.768300,0.007144,0.0,0.648213
std,1.286425,258.877730,0.328094,8.361619,0.038748,0.047273,0.047324,0.481485,0.206732,1.067699e+07,520.000652,1.312382,0.284872,2.326332e+05,871.657022,7.563083,0.005367,0.0,0.105293
min,-5.000000,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,-0.977100,1.000000e+00,1.000000,0.000000,1.250000,3.000000e+00,0.000000,1.000000,0.000031,0.0,0.500002
25%,-0.680000,115.000000,4.101710,12.833333,0.172043,0.207965,0.144219,0.000000,0.050414,7.514925e+06,133.000000,3.000000,3.770000,1.953750e+03,496.000000,1.000000,0.002443,0.0,0.559228
50%,0.170000,226.000000,4.280220,16.173913,0.192953,0.230769,0.165714,0.000000,0.171967,1.756880e+07,287.000000,4.000000,3.960000,7.209000e+03,1029.000000,2.000000,0.005952,0.0,0.630509
75%,0.880000,391.000000,4.474467,20.076923,0.213270,0.256410,0.189130,1.000000,0.304285,2.537215e+07,548.000000,5.000000,4.140000,2.782400e+04,1770.000000,4.000000,0.011173,0.0,0.720883
max,2.920000,3580.000000,12.000000,637.000000,1.000000,1.000000,1.000000,1.000000,0.999500,3.634057e+07,7938.000000,5.000000,5.000000,4.899965e+06,3988.000000,303.000000,0.020000,0.0,0.990272


In [14]:
dat[dat["error"]=="TP"].describe()

,rating_diff,num_words,avg_word_len,avg_sent_len,pct_verbs,pct_nouns,pct_adj,quote,sentiment,book_id,user_reviews_y,user_rating_y,avg_rating,ratings_count,days_since_review_y,review_likes,like_share,popular,predictions
count,42603.000000,42603.000000,42603.000000,42603.000000,42603.000000,42603.000000,42603.000000,42603.000000,42603.000000,4.260300e+04,42603.000000,42603.000000,42603.000000,4.260300e+04,42603.000000,42603.000000,42603.000000,42603.0,42603.000000
mean,-0.081062,349.425440,4.309350,17.216144,0.191799,0.234951,0.168193,0.416074,0.172536,1.729058e+07,597.669296,3.854799,3.935861,9.916865e+03,1202.656902,24.582236,0.118435,1.0,0.727916
std,1.251953,300.405685,0.298044,7.519060,0.034661,0.042021,0.041133,0.492912,0.195739,1.100061e+07,607.492832,1.290726,0.298256,4.887814e+04,888.787799,50.724775,0.162808,0.0,0.128936
min,-5.000000,1.000000,1.750000,1.000000,0.000000,0.000000,0.000000,0.000000,-0.996600,1.000000e+00,1.000000,0.000000,1.250000,3.000000e+00,7.000000,2.000000,0.020013,1.0,0.500005
25%,-0.660000,149.000000,4.124372,13.000000,0.172524,0.210000,0.145205,0.000000,0.048057,7.863218e+06,190.000000,3.000000,3.760000,5.330000e+02,470.000000,4.000000,0.031746,1.0,0.619963
50%,0.160000,276.000000,4.292857,16.360000,0.191919,0.231939,0.164875,0.000000,0.162037,1.822254e+07,405.000000,4.000000,3.950000,1.833000e+03,995.000000,10.000000,0.055503,1.0,0.726802
75%,0.860000,463.000000,4.480327,20.263932,0.210611,0.256039,0.186518,1.000000,0.290107,2.595852e+07,820.000000,5.000000,4.140000,6.174000e+03,1757.000000,26.000000,0.122222,1.0,0.834764
max,2.700000,3524.000000,10.000000,460.500000,0.666667,1.000000,1.000000,1.000000,0.994800,3.643911e+07,7938.000000,5.000000,5.000000,2.758812e+06,3983.000000,3394.000000,1.000000,1.0,0.996426


In [15]:
dat[dat["error"]=="TN"].describe()

,rating_diff,num_words,avg_word_len,avg_sent_len,pct_verbs,pct_nouns,pct_adj,quote,sentiment,book_id,user_reviews_y,user_rating_y,avg_rating,ratings_count,days_since_review_y,review_likes,like_share,popular,predictions
count,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,1.683470e+05,168347.000000,168347.000000,168347.000000,1.683470e+05,168347.000000,168347.000000,168347.000000,168347.0,168347.000000
mean,-0.052876,132.543776,4.223289,14.595006,0.201020,0.227737,0.184634,0.183894,0.226986,1.371871e+07,95.346386,3.981847,4.034723,1.859725e+05,1246.114989,2.594926,0.003986,0.0,0.280408
std,1.122693,172.254198,1.340729,7.779111,0.060533,0.068895,0.075548,0.387399,0.255068,9.934744e+06,161.602942,1.168556,0.271283,5.101134e+05,822.619463,5.758932,0.004650,0.0,0.117190
min,-4.750000,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,-0.988200,1.000000e+00,1.000000,0.000000,1.280000,4.000000e+00,0.000000,1.000000,0.000030,0.0,0.004976
25%,-0.550000,32.000000,3.970000,10.000000,0.171429,0.193798,0.144941,0.000000,0.062987,4.667024e+06,15.000000,3.000000,3.870000,5.900000e+03,600.000000,1.000000,0.000543,0.0,0.184225
50%,0.140000,73.000000,4.191781,13.800000,0.200000,0.222453,0.176471,0.000000,0.220816,1.360243e+07,47.000000,4.000000,4.050000,2.805300e+04,1072.000000,1.000000,0.001908,0.0,0.274947
75%,0.800000,160.000000,4.428571,18.000000,0.228395,0.255814,0.213235,0.000000,0.387150,2.196979e+07,118.000000,5.000000,4.230000,1.204030e+05,1765.000000,2.000000,0.005952,0.0,0.376723
max,2.870000,3634.000000,452.000000,408.000000,1.000000,1.000000,1.000000,1.000000,0.999200,3.641325e+07,4105.000000,5.000000,4.950000,4.899965e+06,4951.000000,483.000000,0.020000,0.0,0.499998
